In [ ]:
import numpy as np
import datetime as dt
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

# download some direct monthly radiation data from cosmo rea6g2

In [ ]:
# !wget https://opendata.dwd.de/climate_environment/REA/COSMO_R6G2/monthly/mon_rsdsdir/rsdsdir_EUR-6km_ECMWF-ERA5_R6G2_r1i1p1f1_COSMOv5_v1_mon_202201-202212.nc

# read the downloaded data

In [ ]:
ifile = 'rsdsdir_EUR-6km_ECMWF-ERA5_R6G2_r1i1p1f1_COSMOv5_v1_mon_202201-202212.nc'
ds = xarray.open_dataset(ifile)
ds

# plotting via xarray backend

## quickplot of data: histogram

In [ ]:
ds['rsdsdir'].plot()

## plot one timestep on map

In [ ]:
ds['rsdsdir'].isel(time=0).plot.pcolormesh(y='latitude', x='longitude',
       subplot_kws=dict(projection=ccrs.Orthographic(), facecolor="gray"),transform=ccrs.PlateCarree())
plt.gca().coastlines();

## plot several timesteps using FacetGrid plotting

In [ ]:
p=ds['rsdsdir'].isel(time=np.arange(4)).plot.pcolormesh(y='latitude', x='longitude', col='time',col_wrap=2,
       subplot_kws=dict(projection=ccrs.Orthographic(), facecolor="gray"),transform=ccrs.PlateCarree())
for ax in p.axs.flat:
    ax.coastlines();

## optimize plotting by creating month as coordinate & dimension

In [ ]:
pdtime = pd.to_datetime(ds['rsdsdir'].time)
ds['month'] = xarray.DataArray(data=pdtime.strftime('%b') + ' 2022', coords={'time':(ds.time.dims,ds.time.values)})
ds = ds.assign_coords(dict(**ds.coords,month=ds.month) )
ds.month

In [ ]:
ds = ds.swap_dims(time='month')
ds

In [ ]:
p=ds['rsdsdir'].isel(month=np.arange(4)).plot.pcolormesh(y='latitude', x='longitude', col='month',col_wrap=2,
       subplot_kws=dict(projection=ccrs.Orthographic(), facecolor="gray"),
       transform=ccrs.PlateCarree())
for ax in p.axs.flat:
    ax.coastlines()

# some statistics

## mean monthly direct radiation in the whole domain

In [ ]:
ds['rsdsdir'].mean(dim=['rlat','rlon']).plot(x='time');

In [ ]:
ds.latitude.min().values, ds.latitude.max().values

## split in latitude bins: mean direct monthly radiation

In [ ]:
ds['rsdsdir'].groupby_bins('latitude', bins=np.arange(20,75,10)).mean(dim=['rlat','rlon']).plot(x='time', hue='latitude_bins');